# Description

Script for doing pose estimation using Intel RealSense D435i, ROS Melodic, OpenCV and ArUco markers

# Imports

In [1]:
import os
import numpy
import cv2
from cv_bridge import CvBridge
from cv_bridge.boost.cv_bridge_boost import getCvType

import rospy
from sensor_msgs.msg import Image, CameraInfo

# Functions

OpenCV Functions

In [2]:
def create_board(config, marker_index):
    '''Creates a aruco board with the given configuration'''
    return cv2.aruco.GridBoard_create(config["markersX"], config["markersY"], config["markerLength"], config["markerSeparation"], config["dictionary"], marker_index)

def create_boards(config):
    '''Creates all four aruco boards and returns a list of them'''
    boards = []
    marker_index = 0
    board_index = 0
    
    for board_index in range(4):
        aruco_board = create_board(board_config, marker_index)
        boards.append(aruco_board)
        marker_index += 4
    return boards

def create_charuco_board(config):
    '''Creates a charuco board with the given configuration'''
    return cv2.aruco.CharucoBoard_create(config["squaresX"], config["squaresY"], config["squareLength"], config["markerLength"], config["dictionary"])

def create_charuco_boards(config):
    '''Creates four charuco boards and return a list of them'''
    charuco_boards = []
    board_index = 0
    
    for board_index in range(4):
        charuco_board = create_charuco_board(config)
        charuco_boards.append(charuco_board)
    
    return charuco_boards

def detect_marker(cv_image):
    '''Detects aruco marker on the given image'''
    aruco_dictionary = cv2.aruco.Dictionary_get(cv2.aruco.DICT_7X7_50)
    markers, ids, _ = cv2.aruco.detectMarkers(cv_image, aruco_dictionary)
    cv2.aruco.drawDetectedMarkers(cv_image, corners=markers,ids=ids, borderColor=150)
    cv2.imshow("Aruco Window", cv_image)
    return cv_image
    
def single_pose_estimate(cv_image):
    '''6DoF Pose estimation of the detected aruco markers'''
    markers, ids, _ = cv2.aruco.detectMarkers(cv_image, board_config["dictionary"])
    if ids is not None:
        if ids.size > 0:
            cv2.aruco.drawDetectedMarkers(cv_image, corners=markers, ids=ids)
            for marker in markers:
                rotation_vector, translation_vector, _ = cv2.aruco.estimatePoseSingleMarkers(corners=marker, markerLength=board_config["markerLength"], cameraMatrix=camera_matrix, distCoeffs=distortion_coefficients)
                cv2.aruco.drawAxis(cv_image, cameraMatrix=camera_matrix, distCoeffs=distortion_coefficients, rvec=rotation_vector, tvec=translation_vector, length=board_config["markerLength"])
    cv2.imshow("Stream", cv_image)
    #out.write(cv_image)

def board_pose_estimation(cv_image):
    '''Does Board pose estimation'''
    markers, ids, _ = cv2.aruco.detectMarkers(cv_image, board_config["dictionary"])
    if ids is not None:
        if ids.size > 0:
            cv2.aruco.drawDetectedMarkers(cv_image, corners=markers, ids=ids)
            valid, rotation_vector, translation_vector = cv2.aruco.estimatePoseBoard(corners=markers, ids=ids, board=boards[0], cameraMatrix=camera_matrix, distCoeffs=distortion_coefficients)
            if valid > 0:
                cv2.aruco.drawAxis(cv_image, cameraMatrix=camera_matrix, distCoeffs=distortion_coefficients, rvec=rotation_vector, tvec=translation_vector)
    cv2.imshow("Board", cv_image)

def charuco_pose_estimation(cv_image):
    '''Does poes estimation using the charuco board (chessboard + aruco marker) in the given configuration'''
    corners, ids, _ = cv2.aruco.detectMarkers(cv_image, charuco_config["dictionary"], parameters=params)
    
    if ids is not None:
        charuco_retval, charuco_corners, charuco_ids = cv2.aruco.interpolateCornersCharuco(corners, ids, cv_image, charuco_boards[0], minMarkers=1)
        cv2.aruco.drawDetectedCornersCharuco(cv_image, charuco_corners, charuco_ids)
        
        retval, rotation_vector, translation_vector = cv2.aruco.estimatePoseCharucoBoard(charuco_corners, charuco_ids, charuco_boards[0],camera_matrix, distortion_coefficients, None, None)# rotation_vector, translation_vector)
        if retval == True:
            cv2.aruco.drawAxis(cv_image, camera_matrix, distortion_coefficients, rotation_vector, translation_vector, 0.03 )
    cv2.imshow("Charuco", cv_image)
    #out.write(cv_image)

ROS Functions

In [3]:
def image_callback(ros_image):
    '''Callback function for the subscription of the ROS topic /camera/color/image_raw (sensor_msgs Image)'''
    global bridge
    cv_image = bridge.imgmsg_to_cv2(ros_image, "bgr8")
    #cv_image = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
    #cv2.imshow("Raw Stream", cv_image)
    #detect_marker(cv_image)
    #single_pose_estimate(cv_image)
    #board_pose_estimation(cv_image)
    charuco_pose_estimation(cv_image)
    cv2.waitKey(1)
    
def calibration_callback(data):
    '''Sets the calibration parameters - camera_matrix and distortion_coefficients - by reading ROS topic /camera/color/cameraInfo'''
    global camera_matrix
    global distortion_coefficients
    camera_matrix = numpy.array(data.K).reshape(3,3)
    distortion_coefficients = numpy.array(data.D).reshape(5,)
    
def run_ros_node():
    '''Initializes the ROS node and setting up Subscriptions and Publishers'''
    rospy.init_node("aruco_estimater", anonymous=True)
    rospy.Subscriber("/camera/color/image_raw", Image, image_callback)
    rospy.Subscriber("/camera/color/camera_info", CameraInfo, calibration_callback)
    rate = rospy.Rate(25)

# Configuration | Instantiation

In [4]:
bridge = CvBridge()
#out = cv2.VideoWriter("200528_charuco_eval_1920_1080_5X5_0.avi", cv2.VideoWriter_fourcc("M", "J", "P", "G"), 30, (640,480))

board_config = {
    "markersX" : 2,
    "markersY" : 2,
    "markerLength" : 0.02, 
    "markerSeparation" :  0.01,
    "margins" : 0.01,
    "dictionary" : cv2.aruco.Dictionary_get(cv2.aruco.DICT_7X7_50),
}

charuco_config = {
    "squaresX" : 5,
    "squaresY" : 5,
    "squareLength" : 0.0115,
    "markerLength" : 0.008,
    "dictionary" : cv2.aruco.Dictionary_get(cv2.aruco.DICT_7X7_50)
}

params = cv2.aruco.DetectorParameters_create()
boards = create_boards(board_config)
charuco_boards = create_charuco_boards(charuco_config)

# P: Projection Camera Matrix (4x3)
# K: Intrinsic Camera Matrix for (distorted) raw images (3x3)
# R: Rectification Matrix (3x3)
# D: Distortion Parameters (5x1)

# Main

In [5]:
def main():
    '''Main Function'''
    
    run_ros_node()
    rospy.spin()
    
    #out.release()

In [ ]:
main()